<a href="https://colab.research.google.com/github/FCosmin27/ocs/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import random

In [12]:
upper_bound = 100
lower_bound = -100
dimension = 10

In [13]:
def check_x(x):
    for i in range(len(x)):
        if x[i] > upper_bound:
            x[i] = upper_bound
        elif x[i] < lower_bound:
            x[i] = lower_bound
    return x

In [14]:
x = np.random.normal(size=dimension)

x = check_x(x)
print(x)

[ 0.15632162  0.55495605 -2.34349392  0.79148215 -0.29618048  0.96514359
 -0.58420348  0.60262036  1.14198474  0.97772021]


In [15]:
def f1(x):
    return np.sum(np.square(x))

In [16]:
def f2(x):
    result = 0
    for i in range(len(x)):
        tmp = 0
        for j in range(i):
            tmp += x[j]

        result += tmp**2

    return result

In [17]:
def f3(x):
    result = 0
    for i in range(1, len(x)):
        result += 1e6 ** ((i-1)/(len(x)-1)) * x[i]**2

    return result

In [18]:
def f4(x):
    result = 0
    for i in range(len(x)):
        tmp = 0
        for j in range(i):
            tmp += x[j]

        result += tmp**2

    return result * (1 + 0.4 * np.abs(np.random.randn()))

In [19]:
def f5(x):
    A = np.random.randint(-500, 501, size=(len(x), len(x)))

    while np.linalg.det(A) == 0:
        A = np.random.randint(-500, 501, size=(len(x), len(x)))

    return np.max(np.abs(x*A - A))

In [ ]:
print("f1(x):", f1(x))
print("f2(x):", f2(x))
print("f3(x):", f3(x))
print("f4(x):", f4(x))
print("f5(x):", f5(x))

In [20]:
def calculate_min_x(func, no_of_agents, x, es, alpha):
    min_x = x
    for _ in range(es):
        xm = min_x + alpha * np.random.normal(size=len(x))
        xm = check_x(xm)
        if func(xm) < func(min_x):
            min_x = xm
    return min_x

In [21]:
def get_population(number_of_variables, population_size):
    return np.random.normal(size=(population_size, number_of_variables))

In [22]:
initial_alpha = 0.5
dimension = 10
population_size = 5
iterations = 100
functions = [f1, f2, f3, f4, f5]
population = get_population(dimension, population_size)

# Lab2

In [33]:
def population_v2_adaptive(population, function, dimension, max_NFE = 1000, initial_alpha = 0.5,):
    NFE = 0
    iter_no = 0

    while NFE < max_NFE:
        iter_no += 1
        alpha = initial_alpha / (iter_no + 1)
        new_population = []

        for agent in population:
            new_agents = [agent + alpha * np.random.normal(size=dimension) for _ in range(5)]
            best_agent = min(new_agents, key=function)
            new_population.append(best_agent)

            NFE += dimension

            if NFE >= max_NFE:
                break

        if sum([function(agent) for agent in new_population]) < sum([function(agent) for agent in population]):
            population = new_population

        if NFE >= max_NFE:
            break

    return population

In [34]:
print("f1(x):", population_v2_adaptive(population, f1, dimension))
print("f2(x):", population_v2_adaptive(population, f2, dimension))
print("f3(x):", population_v2_adaptive(population, f3, dimension))
print("f4(x):", population_v2_adaptive(population, f4, dimension))
print("f5(x):", population_v2_adaptive(population, f5, dimension))

f1(x): [array([ 0.11750775, -0.44404495, -0.21848158, -0.40152405,  0.08502493,
        0.33030404, -0.48885297,  0.65113239,  0.71675038,  0.27460054]), array([ 0.99048162,  1.26568848, -0.38449369,  1.31969432, -0.35320512,
       -0.74774516, -0.22734794,  0.04157135, -0.19442679,  0.11893802]), array([-0.6596545 , -0.84798283,  0.15309119,  0.39784247, -0.37094897,
       -0.10082373,  0.25277788,  0.25331901, -0.19290092,  0.26579353]), array([-0.41696145,  0.28593243,  0.14565162, -1.01382789,  0.42244268,
       -0.82137078,  0.99864761, -1.41629399,  0.80002133,  0.35329819]), array([ 0.11978443, -0.08211577, -0.60943137,  0.05156146, -0.27982093,
        0.30985103,  0.0998803 ,  0.19906729, -0.06598721, -0.20646767])]
f2(x): [array([ 0.47110749, -0.30221489, -0.61419004, -0.56351924,  0.96760253,
       -0.05208055, -0.17976421,  0.58493683,  0.39819158,  0.2955008 ]), array([ 0.9460665 ,  0.67972964, -1.14876697,  2.20706417, -1.05554057,
       -0.8413568 , -1.40159008, -0.

# Lab3

In [36]:
def pc_function(iter_no):
    return max(0.5, 1 / (1 + iter_no))

In [37]:
def pm_function(iter_no):
    return max(0.01, 0.1 / (1 + iter_no))

In [38]:
def sphere_function(x):
    return np.sum(np.square(x))

In [39]:
def CGA_adaptiveV1_greedy(pc_function, pm_function, population, function, dimension, max_NFE = 1000):
    NFE = 0
    iter_no = 0

    while NFE < max_NFE:
        iter_no += 1
        pc = pc_function(iter_no)
        pm = pm_function(iter_no)

        new_population = []
        for i in range(len(population)):
            parent1 = population[i]
            parent2 = population[np.random.randint(len(population))]

            if np.random.rand() < pc:
                crossover_point = np.random.randint(1, dimension - 1)
                child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            else:
                child = parent1.copy()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(dimension)
                child[mutation_idx] += np.random.normal()

            new_population.append(child)

        new_population.sort(key=function)
        best_parent = min(population, key=function)
        new_population[-1] = best_parent

        population = new_population

        NFE += len(new_population)

    return population

In [40]:
final_population = CGA_adaptiveV1_greedy(pc_function, pm_function, population, sphere_function, dimension)
print("Best solution (CGA):", min(final_population, key=sphere_function))


Best solution (CGA): [-0.09196176 -0.14153996 -1.11921004 -0.3453729  -0.91234073  0.23848204
 -0.00652838  0.19366724  0.09208809 -0.80516765]


# Lab4

In [41]:
def pc_function_fitness(fitness):
    return 0.5 + 0.4 * fitness

In [42]:
def pm_function_fitness(fitness):
    return 0.1 - 0.08 * fitness

In [44]:
def RGA_4(population, function, dimension,  pc=0.9, pm=0.1, max_NFE=1000):
    NFE = 0
    while NFE < max_NFE:
        new_population = []
        for _ in range(len(population) // 2):
            parent1, parent2 = population[np.random.choice(len(population), 2, replace=False)]

            if np.random.rand() < pc:
                alpha = 0.1
                child1 = np.random.uniform(np.minimum(parent1, parent2) - alpha * np.abs(parent1 - parent2),
                                           np.maximum(parent1, parent2) + alpha * np.abs(parent1 - parent2))
                child2 = np.random.uniform(np.minimum(parent1, parent2) - alpha * np.abs(parent1 - parent2),
                                           np.maximum(parent1, parent2) + alpha * np.abs(parent1 - parent2))
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(dimension)
                child1[mutation_idx] += np.random.normal()
            if np.random.rand() < pm:
                mutation_idx = np.random.randint(dimension)
                child2[mutation_idx] += np.random.normal()

            child1 = np.clip(child1, -10, 10)
            child2 = np.clip(child2, -10, 10)

            new_population.extend([child1, child2])

        combined_population = np.vstack((population, new_population))
        combined_population = sorted(combined_population, key=function)
        population = np.array(combined_population[:len(population)])

        NFE += len(new_population)

    return population

In [46]:
final_population_v2 = RGA_4(population, f1, dimension)
print("Best solution (RGA_4):", min(final_population_v2, key=f1))

Best solution (RGA_4): [-1.66315272e-02 -1.63946835e-02 -9.98628288e-05 -1.90856114e-03
  9.81301148e-02  1.03507264e-02  3.31637957e-03  1.92526616e-02
  2.01861456e-01 -7.45495757e-03]


# Lab5

In [47]:
def DE_best_1_exp(population, function, F=0.8, CR=0.9, max_NFE=1000):
    NFE = 0
    population_size, dimension = population.shape

    while NFE < max_NFE:
        new_population = []

        for i in range(population_size):
            best = min(population, key=function)
            r1, r2 = population[np.random.choice(population_size, 2, replace=False)]
            mutant = best + F * (r1 - r2)

            trial = np.copy(population[i])
            start_idx = np.random.randint(dimension)
            for j in range(dimension):
                if np.random.rand() < CR or j == start_idx:
                    trial[j] = mutant[j]

            trial = np.clip(trial, -10, 10)

            if function(trial) < function(population[i]):
                new_population.append(trial)
            else:
                new_population.append(population[i])

        population = np.array(new_population)
        NFE += population_size

    return population


In [48]:
final_population_v2 = DE_best_1_exp(population, f1)
print("Best solution (DE_best_1_exp):", min(final_population_v2, key=f1))

Best solution (DE_best_1_exp): [ 0.21656114  0.04685252 -0.05791005  0.20020763 -0.30636745  0.14114458
  0.00542943 -0.00140006  0.0166715  -0.03510926]


## Butnaru Teodor

# Lab2

In [ ]:
def Population_V1_selfAdaptive(nr_iter, nr_agents, start_pt):

    population = [start_pt[:] for _ in range(nr_agents)]
    alfa = [[30, -30] for _ in range(nr_agents)]
    min_value = 0


    for iter in range(nr_iter):
        for i in range(nr_agents):
            xm = [
                population[i][0] + alfa[i][0] * random.uniform(-1, 1),
                population[i][1] + alfa[i][1] * random.uniform(-1, 1)
            ]

            if evaluate_oracle(population[i]) > evaluate_oracle(xm):
                population[i] = xm

            alfa[i][0] += alfa[i][0] * random.uniform(-1, 1)
            alfa[i][1] += alfa[i][1] * random.uniform(-1, 1)

    min_pt = population[0]
    for pt in population:
      if evaluate_oracle(pt)<evaluate_oracle(min_pt):
        min_pt = pt
    return min_pt

nr_iter = 10000
nr_agents = 40
start_pt = [30, -50]

best_solution = Population_V1_selfAdaptive(nr_iter, nr_agents, start_pt)
print(f"Best solution found: {best_solution}")


# Lab3

In [ ]:
def CGA_adaptiveV2_greedy(pc_function, pm_function, population, function, dimension):
    NFE = 0
    max_NFE = 1000

    while NFE < max_NFE:
        fitness_values = np.array([function(individual) for individual in population])
        max_fitness = np.max(fitness_values)
        min_fitness = np.min(fitness_values)

        normalized_fitness = (max_fitness - fitness_values) / (max_fitness - min_fitness + 1e-10)

        pc = pc_function(normalized_fitness)
        pm = pm_function(normalized_fitness)

        new_population = []
        for i in range(len(population)):
            parent1 = population[i]
            parent2 = population[np.random.randint(len(population))]

            if np.random.rand() < pc[i]:
                crossover_point = np.random.randint(1, dimension - 1)
                child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            else:
                child = parent1.copy()

            if np.random.rand() < pm[i]:
                mutation_idx = np.random.randint(dimension)
                child[mutation_idx] += np.random.normal()

            new_population.append(child)

        new_population.sort(key=function)
        best_parent = min(population, key=function)
        new_population[-1] = best_parent

        population = new_population

        NFE += len(new_population)

    return population

# Lab4

In [ ]:
def RGA_3(pc, pm, population, function, no_of_variables):
    NFE = 0
    max_NFE = 1000

    while NFE < max_NFE:
        # Fitness Evaluation
        fitness_values = np.array([function(individual) for individual in population])
        NFE += len(population)

        new_population = []

        # Generate Offspring
        for _ in range(len(population) // 2):
            # Parent Selection
            parent1 = population[np.random.randint(len(population))]
            parent2 = population[np.random.randint(len(population))]

            # Crossover (BLX-0.5)
            if np.random.rand() < pc:
                d = np.abs(parent1 - parent2)
                lower = np.minimum(parent1, parent2) - 0.5 * d
                upper = np.maximum(parent1, parent2) + 0.5 * d
                child1 = np.random.uniform(lower, upper)
                child2 = np.random.uniform(lower, upper)
            else:
                child1, child2 = parent1.copy(), parent2.copy()

            # Mutation
            if np.random.rand() < pm:
                mutation_idx = np.random.randint(no_of_variables)
                child1[mutation_idx] += np.random.normal()

            if np.random.rand() < pm:
                mutation_idx = np.random.randint(no_of_variables)
                child2[mutation_idx] += np.random.normal()

            # Add to new population
            new_population.extend([child1, child2])

        # Replacement
        new_population.sort(key=function)
        population = new_population[:len(population)]

    # Return best solution
    best_solution = min(population, key=function)
    return best_solution, function(best_solution), NFE


In [ ]:
# Run RGA_3
best_solution, best_fitness, nfe_used = RGA_3(
    pc=0.8,
    pm=0.1,
    population=population,
    function=f1,
    no_of_variables=no_of_variables
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Number of Function Evaluations:", nfe_used)

# Lab5

In [ ]:
def DE_rand_to_best_1_exp(F, CR, population, function, no_of_variables):
    NFE = 0
    max_NFE = 1000
    population_size = len(population)

    while NFE < max_NFE:
        fitness_values = np.array([function(ind) for ind in population])
        best_idx = np.argmin(fitness_values)
        best_individual = population[best_idx]
        NFE += len(population)

        new_population = []

        for i in range(population_size):
            # Mutation: rand-to-best/1
            x_rand = population[np.random.randint(population_size)]
            x1, x2 = population[np.random.choice([j for j in range(population_size) if j != i], size=2, replace=False)]
            mutant_vector = x_rand + F * (best_individual - x_rand) + F * (x1 - x2)

            # Crossover: Exponential
            trial_vector = population[i].copy()
            start_index = np.random.randint(no_of_variables)
            length = 0
            while np.random.rand() < CR and length < no_of_variables:
                trial_vector[(start_index + length) % no_of_variables] = mutant_vector[(start_index + length) % no_of_variables]
                length += 1

            # Selection
            if function(trial_vector) < fitness_values[i]:
                new_population.append(trial_vector)
            else:
                new_population.append(population[i])

        population = np.array(new_population)

    # Return best solution
    final_fitness_values = np.array([function(ind) for ind in population])
    best_idx = np.argmin(final_fitness_values)
    return population[best_idx], final_fitness_values[best_idx], NFE


In [ ]:
best_solution, best_fitness, nfe_used = DE_rand_to_best_1_exp(
    F=0.5,
    CR=0.9,
    population=population,
    function=f2,
    no_of_variables=no_of_variables
)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Number of Function Evaluations:", nfe_used)

**Laboratory 6. Testing meta-heuristics**

In [ ]:
D = [5, 10]
NEF = 1000

functions =